## Задача тематического моделирования

что надо было сделать
1. провести исследование и выявить тематики о которых говорят в твитах (для твитов)
2. сделать визуализацию кластеров тематик
3. проинтерпритировать получившиеся тематики

In [ ]:
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE
import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from pprint import pprint

### Подготовка корпуса данных

In [ ]:
#!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0
#!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0

In [1]:
import pandas as pd
import numpy as np

# Загружаем данные из CSV-файлов
positive_df = pd.read_csv('positive.csv', sep=';')
negative_df = pd.read_csv('negative.csv', sep=';')

# Объединяем их в одну выборку
combined_df = pd.concat([positive_df, negative_df], axis=0)

# Сохраняем объединенный датасет в новый CSV-файл, если необходимо
combined_df.to_csv('combined_dataset.csv', index=False)

#combined_df.info()

In [ ]:
# Создание полей для исходного датафрейма
Field_df = pd.DataFrame()
field_template = "Field_{}"

# Задаем количество полей, которые нужно создать
num_fields = combined_df.shape[1]

# Создаем столбцы на основе шаблона
for i in range(1, num_fields + 1):
    field_name = field_template.format(i)
    Field_df[field_name] = None  # Здесь можно указать начальные значения для столбцов

Field_df = list(Field_df)

combined_df.columns = Field_df
combined_df = combined_df.astype('str')

# Берем только твиты и лейблы
combined_df = combined_df[['Field_4', 'Field_5']]

new_column_names = ['tweet', 'label']
combined_df.columns = new_column_names
combined_df.head()

### Предобработка текста

In [ ]:
selected_columns = combined_df
selected_columns.head()

In [ ]:
import nltk
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import re

# Инициализируйте библиотеки nltk и pymorphy2
nltk.download('stopwords')
nltk.download('punkt')
morph = MorphAnalyzer()
stop_words = set(stopwords.words("russian"))

# Пример твита
tweet = "Написал код для обработки текстовых данных в Python! #python #NLP"

# Функция для предобработки текста
def preprocess_tweet(tweet):
    if isinstance(tweet, str):
        # Токенизация: разделение текста на слова или токены
        words = nltk.word_tokenize(tweet)
        
        # Приведение к нижнему регистру
        words = [word.lower() for word in words]
        
        # Удаление знаков препинания и цифр
        words = [re.sub(r'[^а-яА-Яa-zA-Z]', '', word) for word in words]
        
        # Удаление стоп-слов
        words = [word for word in words if word not in stop_words]
        
        # Лемматизация: приведение слов к их базовой форме
        words = [morph.parse(word)[0].normal_form for word in words]
        
        return ' '.join(words)

    else:
        return words

# Применение предобработки к твиту
preprocessed_tweet = preprocess_tweet(tweet)
print(preprocessed_tweet)

In [ ]:
selected_columns = selected_columns.dropna(subset=['tweet'])
selected_columns['tweet'] = selected_columns['tweet'].astype(str)
selected_columns['tweet'] = selected_columns['tweet'].apply(preprocess_tweet)
#selected_columns.head(1)
selected_columns.to_csv('tweets_for_analysis.csv', index=False)

### Тематическое моделирование

In [2]:
data = pd.read_csv('tweets_for_analysis.csv')
data['tweet'].fillna('', inplace=True)
#data.info()

# Список стоп-слов на русском языке
from stop_words import get_stop_words
stop_words = get_stop_words('russian')

In [ ]:
# Создание мешка слов (Bag of Words)
vectorizer = CountVectorizer(max_df=0.8, min_df=2, stop_words=stop_words)
X = vectorizer.fit_transform(data['tweet'])

# Обучение модели LDA
num_topics = 10  # Задайте количество тем
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)

# Вывод топ-слов для каждой темы
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = f"Тема #{topic_idx + 1}: "
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

n_top_words = 10  # Количество топ-слов для каждой темы
print("\nТоп слова для каждой темы:")
print_top_words(lda, vectorizer.get_feature_names_out(), n_top_words)

# Визуализация кластеров тематик с помощью t-SNE
tsne = TSNE(n_components=2, random_state=42)
tsne_representation = tsne.fit_transform(lda.transform(X))

# Создание графика
plt.figure(figsize=(10, 8))
plt.scatter(tsne_representation[:, 0], tsne_representation[:, 1], c=lda.predict(X), cmap='viridis')
plt.title('Визуализация кластеров тематик')
plt.colorbar()
plt.show()

In [3]:
# Обучение модели LDA
num_topics = 5  # Количество тем
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)

# Вывод топ-слов для каждой темы
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = f"Тема #{topic_idx + 1}: "
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

n_top_words = 10  # Количество топ-слов для каждой темы
print("\nТоп слова для каждой темы:")
print_top_words(lda, vectorizer.get_feature_names_out(), n_top_words)


Топ слова для каждой темы:
Тема #1: http rt любить ахи ахах свой знать новый ахахи понять
Тема #2: завтра написать пойти спать rt писать школа ночь вс идти
Тема #3: http rt знать делать думать смотреть любить свой слушать видеть
Тема #4: rt http новый привет фильм любить друг нравиться смотреть говорить
Тема #5: http rt хороший утро добрый настроение рождение вечер самый дом


Исходя из топ слов для каждой темы, можно предположить следующие интерпретации:

1. **Тема #1: Эмоции и отношения**
   - Эта тема может быть связана с выражением эмоций, в том числе радости (ахи, ахах), любви (любить), новых впечатлений (новый), и понимания (понять).

2. **Тема #2: Планы и активности**
   - В этой теме вероятно рассматриваются различные активности и планы, такие как написание (написать), пойти (пойти), школа (школа), и ночь (ночь).

3. **Тема #3: Размышления и восприятие**
   - Эта тема может относиться к размышлениям (думать), восприятию (видеть), пониманию (знать), и отношениям (любить, свой).

4. **Тема #4: Развлечения и общение**
   - Здесь, вероятно, рассматриваются различные развлечения и общение, такие как фильмы (фильм, смотреть), новые знакомства (новый, привет), и общение (говорить).

5. **Тема #5: Настроение и время суток**
   - Эта тема, возможно, связана с описанием настроения (настроение, хороший, утро, добрый) и различными временами суток (вечер, дом).